# EA Two
Just doing some recleaning and maybe simple regression

In [31]:
import os

import matplotlib.pyplot as plt

import numpy as np

import folium
import geopandas as gpd
from folium.plugins import HeatMap

import pandas as pd
import json
from json import loads, dumps

In [32]:
measures_df = pd.read_csv("./HCAHPSPatientSurveyFiles/data_tables/measures.csv")
reports_df = pd.read_csv('./HCAHPSPatientSurveyFiles/data_tables/reports.csv')
national_results_df = pd.read_csv('./HCAHPSPatientSurveyFiles/data_tables/national_results.csv')
questions_df = pd.read_csv('./HCAHPSPatientSurveyFiles/data_tables/questions.csv')
responses_df = pd.read_csv('./HCAHPSPatientSurveyFiles/data_tables/responses.csv')
state_results_df = pd.read_csv('./HCAHPSPatientSurveyFiles/data_tables/state_results.csv')
states_df = pd.read_csv('./HCAHPSPatientSurveyFiles/data_tables/states.csv')

In [33]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)
path = "./HCAHPSPatientSurveyFiles/data_tables/"

In [34]:
##For future reference put in list and iterate through list.  Might save on lines of code but unsure of speed
#Strip Whitespace from columns
measures_strip = measures_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
national_results_strip_df = national_results_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
questions_strip_df = questions_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
reports_strip_df = reports_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
responses_strip_df = responses_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
states_strip_df = states_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
state_results_strip_df = state_results_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [60]:
states_strip_df.head()

,State,State Name,Region
0,AK,Alaska,Pacific
1,AL,Alabama,East South Central
2,AR,Arkansas,West South Central
3,AZ,Arizona,Mountain
4,CA,California,Pacific


In [36]:
##Converting to date/time results in adding the 1st day of the month.
national_results_strip_df['Release Period'] = pd.to_datetime(national_results_strip_df['Release Period'], format='%m_%Y')

In [37]:
responses_strip_df['Release Period'] = pd.to_datetime(responses_strip_df['Release Period'], format='%m_%Y')

In [38]:
state_results_strip_df['Release Period'] = pd.to_datetime(state_results_strip_df['Release Period'], format='%m_%Y')

In [39]:
responses_strip_df['Completed Surveys'].replace({"FEWER THAN 50": '25', "300 or more": '300',
                                                   "Between 100 and 299":'250',"Fewer than 100":'50'}, inplace=True)

not_available = responses_strip_df[ (responses_strip_df['Response Rate (%)'] == 'Not Available')].index
responses_strip_df.drop(not_available , inplace=True)

In [40]:
#convert Completed Surveys string to int64
responses_strip_df['Completed Surveys'] = responses_strip_df['Completed Surveys'].astype(np.int64)

In [41]:
##Convert Response Rate to int64
responses_strip_df['Response Rate (%)'] = responses_strip_df['Response Rate (%)'].astype(np.int64)

In [42]:
state_results_strip_df.head()

,Release Period,State,Measure ID,Bottom-box Percentage,Middle-box Percentage,Top-box Percentage
0,2015-07-01,AK,H_CLEAN_HSP,8,22,70
1,2015-07-01,AK,H_COMP_1,9,17,74
2,2015-07-01,AK,H_COMP_2,10,15,75
3,2015-07-01,AK,H_COMP_3,11,21,68
4,2015-07-01,AK,H_COMP_5,19,17,64


In [43]:
responses_strip_df.head()

,Release Period,State,Facility ID,Completed Surveys,Response Rate (%)
0,2015-07-01,AL,10001,300,27
1,2015-07-01,AL,10005,300,37
2,2015-07-01,AL,10006,300,25
3,2015-07-01,AL,10007,250,30
4,2015-07-01,AL,10008,50,28


In [44]:
response_states = pd.merge(state_results_strip_df, responses_strip_df, on=['Release Period', 'State'], how='outer')

In [45]:
response_regionState = pd.merge(
    states_strip_df, response_states, how="outer", on=["State"])

In [61]:
response_regionState.head()

,State,State Name,Region,Release Period,Measure ID,Bottom-box Percentage,Middle-box Percentage,Top-box Percentage,Facility ID,Completed Surveys,Response Rate (%)
0,AK,Alaska,Pacific,2015-07-01,H_CLEAN_HSP,8.0,22.0,70.0,20001,300,24
1,AK,Alaska,Pacific,2015-07-01,H_CLEAN_HSP,8.0,22.0,70.0,20006,300,34
2,AK,Alaska,Pacific,2015-07-01,H_CLEAN_HSP,8.0,22.0,70.0,20008,250,26
3,AK,Alaska,Pacific,2015-07-01,H_CLEAN_HSP,8.0,22.0,70.0,20012,300,28
4,AK,Alaska,Pacific,2015-07-01,H_CLEAN_HSP,8.0,22.0,70.0,20017,300,40


In [47]:
filtered_df = response_regionState[response_regionState['State'] == 'VI'] ##'PR'
filtered_df

,State,State Name,Region,Release Period,Measure ID,Bottom-box Percentage,Middle-box Percentage,Top-box Percentage,Facility ID,Completed Surveys,Response Rate (%)
374030,VI,NaN,NaN,2015-07-01,NaN,NaN,NaN,NaN,480001,250,35
374031,VI,NaN,NaN,2015-07-01,NaN,NaN,NaN,NaN,480002,300,27
374032,VI,NaN,NaN,2016-07-01,NaN,NaN,NaN,NaN,480001,287,37
374033,VI,NaN,NaN,2016-07-01,NaN,NaN,NaN,NaN,480002,435,27
374034,VI,NaN,NaN,2017-07-01,NaN,NaN,NaN,NaN,480001,564,35
374035,VI,NaN,NaN,2017-07-01,NaN,NaN,NaN,NaN,480002,508,29
374036,VI,NaN,NaN,2018-07-01,NaN,NaN,NaN,NaN,480001,472,30
374037,VI,NaN,NaN,2018-07-01,NaN,NaN,NaN,NaN,480002,420,26
374038,VI,NaN,NaN,2020-07-01,NaN,NaN,NaN,NaN,480001,595,31
374039,VI,NaN,NaN,2020-07-01,NaN,NaN,NaN,NaN,480002,320,26


In [48]:
rows_with_nan = response_states[response_regionState.isnull().any(axis=1)]

print(rows_with_nan)

       Release Period State Measure ID  Bottom-box Percentage  \
374030     2015-07-01    VI        NaN                    NaN   
374031     2015-07-01    VI        NaN                    NaN   
374032     2016-07-01    VI        NaN                    NaN   
374033     2016-07-01    VI        NaN                    NaN   
374034     2017-07-01    VI        NaN                    NaN   
374035     2017-07-01    VI        NaN                    NaN   
374036     2018-07-01    VI        NaN                    NaN   
374037     2018-07-01    VI        NaN                    NaN   
374038     2019-07-01    PR        NaN                    NaN   
374039     2020-07-01    PR        NaN                    NaN   
374040     2020-07-01    PR        NaN                    NaN   
374041     2020-07-01    PR        NaN                    NaN   
374042     2020-07-01    PR        NaN                    NaN   
374043     2020-07-01    VI        NaN                    NaN   
374044     2020-07-01    

In [49]:
#this removes PR from the list due to PR having no state or region.  The row count drops to 374030 from 374072
#what is the ethical value of dropping PR as it has missing data.  The same applies to VI
##I decided to drop VI (Virgin Island) and PR (Puerto Rico).  Both datasets can be used however.
response_regionState_clean = response_regionState.dropna()

In [62]:
response_regionState_clean.head()

,State,State Name,Region,Release Period,Measure ID,Bottom-box Percentage,Middle-box Percentage,Top-box Percentage,Facility ID,Completed Surveys,Response Rate (%)
0,AK,Alaska,Pacific,2015-07-01,H_CLEAN_HSP,8.0,22.0,70.0,20001,300,24
1,AK,Alaska,Pacific,2015-07-01,H_CLEAN_HSP,8.0,22.0,70.0,20006,300,34
2,AK,Alaska,Pacific,2015-07-01,H_CLEAN_HSP,8.0,22.0,70.0,20008,250,26
3,AK,Alaska,Pacific,2015-07-01,H_CLEAN_HSP,8.0,22.0,70.0,20012,300,28
4,AK,Alaska,Pacific,2015-07-01,H_CLEAN_HSP,8.0,22.0,70.0,20017,300,40


In [59]:
filtered_df = response_regionState_clean[response_regionState_clean['State'] == 'AK'].sort_values(by=['Release Period',
                                                                                                      'Facility ID',
                                                                                                    'Measure ID'])
state_region_r

,State,State Name,Region,Release Period,Measure ID,Bottom-box Percentage,Middle-box Percentage,Top-box Percentage,Facility ID,Completed Surveys,Response Rate (%)
0,AK,Alaska,Pacific,2015-07-01,H_CLEAN_HSP,8.0,22.0,70.0,20001,300,24
16,AK,Alaska,Pacific,2015-07-01,H_COMP_1,9.0,17.0,74.0,20001,300,24
32,AK,Alaska,Pacific,2015-07-01,H_COMP_2,10.0,15.0,75.0,20001,300,24
48,AK,Alaska,Pacific,2015-07-01,H_COMP_3,11.0,21.0,68.0,20001,300,24
64,AK,Alaska,Pacific,2015-07-01,H_COMP_5,19.0,17.0,64.0,20001,300,24
...,...,...,...,...,...,...,...,...,...,...,...
1197,AK,Alaska,Pacific,2023-07-01,H_COMP_6,14.0,0.0,86.0,21313,121,19
1210,AK,Alaska,Pacific,2023-07-01,H_COMP_7,5.0,46.0,49.0,21313,121,19
1223,AK,Alaska,Pacific,2023-07-01,H_HSP_RATING,7.0,22.0,71.0,21313,121,19
1236,AK,Alaska,Pacific,2023-07-01,H_QUIET_HSP,11.0,30.0,59.0,21313,121,19


In [52]:
national_results_strip_df.head()

,Release Period,Measure ID,Bottom-box Percentage,Middle-box Percentage,Top-box Percentage
0,2015-07-01,H_CLEAN_HSP,8,18,74
1,2015-07-01,H_COMP_1,4,17,79
2,2015-07-01,H_COMP_2,4,14,82
3,2015-07-01,H_COMP_3,9,23,68
4,2015-07-01,H_COMP_5,18,17,65


In [63]:
questions_strip_df.head()

,Question Num,Measure ID,Question,Bottom-box Answer,Middle-box Answer,Top-box Answer
0,1,H_COMP_1,"During this hospital stay, how often did nurses treat you with courtesy and respect?",Sometimes or never,Usually,Always
1,2,H_COMP_1,"During this hospital stay, how often did nurses listen carefully to you?",Sometimes or never,Usually,Always
2,3,H_COMP_1,"During this hospital stay, how often did nurses explain things in a way you could understand?",Sometimes or never,Usually,Always
3,4,H_COMP_3,"During this hospital stay, after you pressed the call button, how often did you get help as soon as you wanted it?",Sometimes or never,Usually,Always
4,5,H_COMP_2,"During this hospital stay, how often did doctors treat you with courtesy and respect?",Sometimes or never,Usually,Always


In [55]:
reports_strip_df

,Release Period,Start Date,End Date
0,07_2015,2013-10-01,2014-09-30
1,07_2016,2014-10-01,2015-09-30
2,07_2017,2015-10-01,2016-09-30
3,07_2018,2016-10-01,2017-09-30
4,07_2019,2017-10-01,2018-09-30
5,07_2020,2018-10-01,2019-09-30
6,07_2021,2019-10-01,2020-09-30
7,07_2022,2020-10-01,2021-09-30
8,07_2023,2021-10-01,2022-09-30


In [57]:
merged_df = pd.merge(national_results_strip_df, responses_strip_df, on=['Release Period'])

In [58]:
merged_df

,Release Period,Measure ID,Bottom-box Percentage,Middle-box Percentage,Top-box Percentage,State,Facility ID,Completed Surveys,Response Rate (%)
0,2015-07-01,H_CLEAN_HSP,8,18,74,AL,10001,300,27
1,2015-07-01,H_CLEAN_HSP,8,18,74,AL,10005,300,37
2,2015-07-01,H_CLEAN_HSP,8,18,74,AL,10006,300,25
3,2015-07-01,H_CLEAN_HSP,8,18,74,AL,10007,250,30
4,2015-07-01,H_CLEAN_HSP,8,18,74,AL,10008,50,28
...,...,...,...,...,...,...,...,...,...
374445,2023-07-01,H_RECMND,6,25,69,TX,670143,42,28
374446,2023-07-01,H_RECMND,6,25,69,TX,670259,34,34
374447,2023-07-01,H_RECMND,6,25,69,TX,670260,454,14
374448,2023-07-01,H_RECMND,6,25,69,TX,670300,186,15


In [30]:
merged_df[(merged_df['State'] == 'AL') & (merged_df['Release Period'] == '07_2015')]

,Release Period,Measure ID,Bottom-box Percentage,Middle-box Percentage,Top-box Percentage,State,Facility ID,Completed Surveys,Response Rate (%)
